In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install -r /content/drive/MyDrive/Project/requirements.txt

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 115.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.5 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=c8c59a7967955a092bdab2d130636f8670876e5449e18ffb7b2f1ce5020674a9
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
  Attempting uninstall: SQLAlchemy
    Found existing ins

#PREPARE DB

In [2]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq

In [6]:
from google.colab import userdata
GROQ = userdata.get('GROQ_API')

In [7]:
import os
os.environ['GROQ_API_KEY'] = GROQ

In [8]:
data_path = '/content/drive/MyDrive/Project/data/'
vector_db_path = 'vector_db/'
EMBEDDING_PATH = '/content/drive/MyDrive/Project/models/all-MiniLM-L6-v2.F16.gguf'


In [9]:
from langchain.embeddings import GPT4AllEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

def create_db(persist_directory):
  loader = DirectoryLoader(data_path, glob='*.pdf', loader_cls=PyPDFLoader)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(
      separators = '\n',
      chunk_size = 500,
      chunk_overlap = 50,
      length_function = len
  )
  embedding = GPT4AllEmbeddings(model=EMBEDDING_PATH)
  chunks = text_splitter.split_documents(documents)
  db = Chroma.from_documents(chunks, embedding, persist_directory=persist_directory)
  db.persist()
  return db

def setUp_retriever(db):
  retriever = db.as_retriever()
  return retriever

#QA

In [10]:
from langchain_community.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [16]:
def load_llm():
    llm = ChatGroq(
        groq_api_key = GROQ,
        model_name = 'Llama3-8b-8192',
        temperature=0
    )
    return llm

def create_prompt(template):
  prompt = PromptTemplate(template=template, input_variables=["context","question"])
  return prompt

def create_chain(llm, prompt, db):
  retriever = db.as_retriever(
      search_type="similarity_score_threshold",
        search_with_scores=False,
        search_kwargs={"score_threshold": 0.45, "k": 3},
    )
  chain = RetrievalQA.from_chain_type(
      llm=llm,
      chain_type='stuff',
      retriever=retriever,
      return_source_documents=True,
      chain_type_kwargs={'prompt': prompt}
  )
  return chain

def give_answer(chain, question):
  res = chain.invoke({'query': question})
  #get the relevent infor

  return res


def extract_answer(res):
  # Instead of accessing a dictionary, we return the result string itself
  # or you can process the result string further to extract the answer
  # based on the expected format of your chain's output.
  answer = res # Assuming the answer is directly in the response
  return answer

In [17]:
pip install chromadb

In [19]:
persist_directory = 'vector_db'
db = create_db(persist_directory)
llm = load_llm()

template = '''<|im_start|>system
Bạn là trợ lý AI hũu ích của trường Đại học Công nghệ - Đại học Quốc gia Hà Nội.Dựa vào thông tin cung cấp hãy đưa ra thông tin hữu ích cho sinh viên. Hãy đưa ra kết quả là Tiếng Việt.
Nếu không tìm được câu trả lời thì không đưa ra kết quả.
{context}<|im_end|><|im_start|>user
{question}<|im_end|>
<|im_start|>assistant'''


prompt = create_prompt(template)
chain = create_chain(llm, prompt, db)

In [22]:
question = 'Hãy cho tôi thông tin địa chỉ Trường Đại học Quốc Gia Hà Nội'
give_answer(chain, question)

{'query': 'Hãy cho tôi thông tin địa chỉ Trường Đại học Quốc Gia Hà Nội',
 'result': 'Xin chào sinh viên!\n\nTheo thông tin cung cấp, Trường Đại học Công nghệ - Đại học Quốc gia Hà Nội có địa chỉ tại:\n\nĐịa chỉ: 136 Xóm Dịch, Phường Nam Từ Liêm, Quận Bắc Từ Liêm, Hà Nội\n\nVui lòng ghi nhớ địa chỉ này để dễ dàng đến trường và tham gia các hoạt động học tập. Nếu có bất kỳ thắc mắc nào, vui lòng liên hệ với chúng tôi để được hỗ trợ.\n\nChúc các bạn học tập tốt!',
 'source_documents': [Document(metadata={'page': 5, 'source': '/content/drive/MyDrive/Project/data/uet-vnu-edu-vn-lich-su-.pdf'}, page_content='Viễn thông, Khoa học Máy tính, Công nghệ Kỹ thuật – Cơ điện tử, Mạng máy tính và Truyền\nthông Dữ liệu) được kiểm định theo bộ tiêu chuẩn AUN-QA của Mạng lưới các Trường Đại học\nĐông Nam Á.\nTừ ngày 8/11 – 12/11/2022, Đoàn đánh giá ngoài đến từ Trung tâm Kiểm định chất lượng giáo\ndục, Đại học Quốc gia Thành phố Hồ Chí Minh (CEA) đã hoàn thành đợt khảo sát chính thức,\nđánh giá chất lư